In [20]:
import json
import re
from pathlib import Path
import glob
from PIL import Image
import numpy as np

import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from nomic import atlas

from PIDDetectionEncoder import ObjectEncoder, PIDObjects, AugmentedViewGenerator, img_augmentation
from PIDObjectLookupTable import ObjectLookupTable

In [4]:
root = Path().resolve().parents[1]
img_dir = root / 'Data' / 'SimCLR'
    
# Image Augmentation pipeline
augmentation_pipeline = img_augmentation(img_size=224, blur_kernel_size=3)
# Generates 2 augmented samples for each img
augmentation_generator = AugmentedViewGenerator(augmentation_pipeline, n_views=8)
# 40 P&ID symbols with 2 randomly augmented views of each
pid_dataset = PIDObjects(img_dir=img_dir, transform=augmentation_generator)

train_dataloader = DataLoader(
    pid_dataset,
    batch_size=108,
    shuffle=True
)

c:\Users\Andrew Deur\miniconda3\envs\capstone\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [5]:
class UnnormalizePIDObjects():

    def __init__(self):
        self.mean = torch.tensor([0.753, 0.753, 0.754]).reshape(-1, 1, 1)
        self.std = torch.tensor([0.333, 0.333, 0.332]).reshape(-1, 1, 1)
   
    def unnormalize(self, tensor):
        unnormalized_tensor = tensor * self.std + self.mean
        return unnormalized_tensor

In [6]:
for batch, labels in train_dataloader:


    to_pil = transforms.ToPILImage()
    unnormalizer = UnnormalizePIDObjects()

    unnormalized_batch = [unnormalizer.unnormalize(x) for x in batch]
     
    pil_imgs = [
        [to_pil(x[i]) for i in range(x.size(0))]
        for x in unnormalized_batch 
    ]

    img_suffix_counter = {}

    for batch in pil_imgs:
        for i, img in enumerate(batch):
            
            label = labels[i].item()
            
            # Init value @ -1 b/c value is offset from 'A' ascii id
            img_suffix_counter[label] = img_suffix_counter.get(label, 64) 

            # Skip past non printable ascii chars (91-96)
            img_suffix_counter[label] = img_suffix_counter[label] + (1 if img_suffix_counter[label] != 90 else 7)
            
            save_path = root / 'Data' / 'SimCLR_augmentations' / f'{labels[i]}_{chr(img_suffix_counter[labels[i].item()])}.png'
            print(save_path)
            img.save(save_path)

C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\3_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\20_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\7_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\16_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\20_B.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\1_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\11_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\31_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\30_A.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\1_B.png
C:\Users\Andrew Deur\Documents\NYU\DS-GA 1006 Capstone\Data\SimCLR_augmentations\33_A.png
C:\Users\Andre

In [70]:
root = Path().resolve().parents[1]
img_dir = root / 'Data' / 'SimCLR_augmentations' / '*.png'
imgs = glob.glob(str(img_dir))

# Pre-Trained Encoder Path
encoder_pth = root / 'nyu-capstone-2024-PIDGraph' / 'Object Detection'/ 'models' / 'encoder_1.pth'

encoder = torch.load(encoder_pth)
lookup_tables = ObjectLookupTable(labels, imgs, encoder, img_size=224)

C:\Users\Andrew Deur\AppData\Local\Temp\ipykernel_4360\3195488443.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder = torch.load(encoder_pth)
c:\Users\Andrew Deur\

In [71]:
embeddings_np = lookup_tables.gt_embeddings.cpu().numpy()

In [72]:
class_labels = root / 'nyu-capstone-2024-PIDGraph' / 'Object Detection' / 'classes_general.json'
with open(class_labels) as json_data:
    d = json.load(json_data)

In [76]:
img_classes = [
    str(int(re.findall('(\d+)(?:_[a-z]+)?\.png', path, re.IGNORECASE)[0]))
    for path in imgs
]

text_labels = [d[i]['class_name'] for i in img_classes]

In [79]:
data = [
    {
        "id": f"img_{i}",
        "label": text_labels[i],
        "image_path": img_path
    }
    for i, img_path in enumerate(imgs)
]

In [80]:
atlas.map_data(
    data=data,
    blobs=imgs,
    embeddings=embeddings_np,
    identifier='P&ID Object Embedding Clustering (Resnet18 Backbone)'
)

2024-12-17 20:19:44.299 | WARNING  | nomic.atlas:map_data:128 - An ID field was not specified in your data so one was generated for you in insertion order.
2024-12-17 20:19:46.057 | INFO     | nomic.dataset:_create_project:867 - Organization name: `ad3254`
2024-12-17 20:19:46.458 | INFO     | nomic.dataset:_create_project:895 - Creating dataset `pid-object-embedding-clustering-resnet18-backbone`
2024-12-17 20:19:46.799 | INFO     | nomic.atlas:map_data:140 - Uploading data to Atlas.
Loading images: 100%|██████████| 791/791 [00:00<00:00, 1059.68it/s]
Uploading blobs to Atlas: 792it [00:18, 43.88it/s]                          
2024-12-17 20:20:05.612 | INFO     | nomic.atlas:map_data:158 - `ad3254/pid-object-embedding-clustering-resnet18-backbone`: Data upload succeeded to dataset`
2024-12-17 20:20:05.866 | WARNING  | nomic.dataset:create_index:1139 - Ignoring indexed_field for image datasets. Only _blob_hash is supported.
2024-12-17 20:20:06.752 | INFO     | nomic.dataset:create_index:1

AtlasDataset: <{'id': '2bcdf41e-0134-4639-a2f3-d3fed348df18', 'owner': '8058a4ad-c547-4191-86ed-4d878eea0820', 'project_name': 'pid-object-embedding-clustering-resnet18-backbone', 'creator': 'google-oauth2|103154032702218896252', 'description': '', 'is_public': True, 'is_public_to_org': True, 'project_fields': ['id', 'label', 'image_path', 'id_', '_blob_hash'], 'unique_id_field': 'id_', 'modality': 'image', 'total_datums_in_project': 791, 'created_timestamp': '2024-12-18T01:19:44.311326+00:00', 'slug': 'pid-object-embedding-clustering-resnet18-backbone', 'connector_name': None, 'organization_slug': 'ad3254', 'organization_name': 'ad3254', 'creator_nickname': 'ad3254', 'creator_picture': 'https://lh3.googleusercontent.com/a/ACg8ocKOBsL9r4coghud8oaDP3I3TZnHvtk-8mvCoRGawlibcLBKPQ=s96-c', 'og_image': 'https://atlas-content-production.s3.amazonaws.com/projects/2bcdf41e-0134-4639-a2f3-d3fed348df18/og-map.png', 'atlas_indices': [{'id': '80f4bc17-a9eb-4c3d-bee0-683823bdb8ad', 'project_id': '2bcdf41e-0134-4639-a2f3-d3fed348df18', 'index_name': 'P&ID Object Embedding Clustering (Resnet18 Backbone)', 'indexed_field': '_blob_hash', 'created_timestamp': '2024-12-18T01:20:04.246924+00:00', 'updated_timestamp': '2024-12-18T01:20:04.246924+00:00', 'atoms': ['charchunk', 'document'], 'colorable_fields': ['id', 'label', 'image_path'], 'embedders': [{'id': '5228f255-d793-40cc-a3e7-fcbe2b6eabd7', 'atlas_index_id': '80f4bc17-a9eb-4c3d-bee0-683823bdb8ad', 'ready': False, 'model_name': 'nomic-embed-vision-v1.5', 'hyperparameters': {'norm': 'both', 'batch_size': 20, 'polymerize_by': 'charchunk', 'dataset_buffer_size': 1000}}], 'nearest_neighbor_indices': [{'id': '59e54419-ebab-4caf-8910-325db077da33', 'index_name': 'NomicOrganize', 'ready': False, 'hyperparameters': {'M': 16, 'space': 'l2', 'ef_construction': 100}, 'atom_strategies': ['document']}], 'projections': [{'id': 'fc39a0b5-5054-4356-b097-d5c74b6a3603', 'projection_name': 'NomicProject', 'ready': False, 'hyperparameters': {'rho': 0.0, 'model': 'nomic-project-v2', 'spread': 1, 'n_noise': None, 'min_dist': 0.4, 'n_epochs': 50, 'n_neighbors': 5, 'n_init_epochs': 20, 'local_neighborhood_size': 15}, 'atom_strategies': ['document'], 'created_timestamp': '2024-12-18T01:20:04.262597+00:00', 'updated_timestamp': '2024-12-18T01:20:04.262597+00:00', 'thumbnail': 'https://atlas-content-production.s3.amazonaws.com/projects/2bcdf41e-0134-4639-a2f3-d3fed348df18/fc39a0b5-5054-4356-b097-d5c74b6a3603/map.png', 'og_image': 'https://atlas-content-production.s3.amazonaws.com/projects/2bcdf41e-0134-4639-a2f3-d3fed348df18/fc39a0b5-5054-4356-b097-d5c74b6a3603/og-map.png'}]}], 'insert_update_delete_lock': True, 'access_role': 'ADMIN', 'permissions': {'dataset:administration:read': True, 'dataset:administration:write': True, 'dataset:metadata:read': True, 'dataset:metadata:write': True, 'dataset:data:read': True, 'dataset:data:write': True, 'dataset:data:add': True, 'dataset:data:delete': True, 'dataset:resource:create': True, 'dataset:resource:delete': True, 'dataset:members:read': True, 'dataset:members:write': True, 'dataset:tags:read': True, 'dataset:tags:write': True}, 'schema': '/////4ABAAAQAAAAAAAKAA4ABgAFAAgACgAAAAABBAAQAAAAAAAKAAwAAAAEAAgACgAAAGAAAAAEAAAAAQAAAAwAAAAIAAwABAAIAAgAAAA0AAAABAAAACQAAAAyYmNkZjQxZS0wMTM0LTQ2MzktYTJmMy1kM2ZlZDM0OGRmMTgAAAAACgAAAHByb2plY3RfaWQAAAUAAADIAAAAjAAAAFwAAAA0AAAABAAAAFz///8AAAEFEAAAABwAAAAEAAAAAAAAAAoAAABfYmxvYl9oYXNoAABU////iP///wAAAQUQAAAAFAAAAAQAAAAAAAAAAwAAAGlkXwB4////rP///wAAAQUQAAAAHAAAAAQAAAAAAAAACgAAAGltYWdlX3BhdGgAAKT////Y////AAABBRAAAAAYAAAABAAAAAAAAAAFAAAAbGFiZWwAAADM////EAAUAAgABgAHAAwAAAAQABAAAAAAAAEFEAAAABgAAAAEAAAAAAAAAAIAAABpZAAABAAEAAQAAAA=', 'connector_details': None}>